In our radCAD model, traders will initiate a long position at timestep 0 with `trade_leverage_ratio` degree of leverage. Each timestep, the open trade will incur interest rate charges on any borrowed funds to finance the trade (ie if leverage is >1), and track running profit/loss based on the simulated price changes we will generate from the brownian_motion_generator repo.

This yields an expected Profit/Loss for each simulation, and we can compare expected results for different leverage ratios and borrowing costs. The benefit of using our simulated random walks is that we can run a Monte Carlo simulation on this simple trading system, to see what is likely to happen in a variety of cases.

For example, a higher leverage ratio may be more profitable *on average* up to a point vs a lower leverage ratio, but it will also get liquidated more often. Thus, we can optimize for more complex outcomes than simply maximizing profit - we can optimize for profit relative to risk for example.

This is especially relevant for protocol builders, which need to always consider what *may* happen and the relative risks, not just what is expected to happen *on average*. For example, USDT held its peg on the average day - but it's obviously meaningless to say a protocol "works on average" if it works 80% of the time but fatally explodes 20% of the time. In USDT's case, the system 's design had fatal flaws that analysts using techniques like Monte Carlo analysis identified were able to identify ahead of the crash. This doens't mean Monte Carlo can predict *when* a system will fail, but it can demonstrate if a failure is likely, and inevitable to eventually occur.

This simple tutorial contains the building blocks for extending to simulating and modeling risks for any protocols making use of collateral and/or leverage such as Maker DAO, Aave, and more.


In [1]:
import requests
import json
import pandas as pd
import numpy as np
import brownian_motion_generator as bmg # Assumes brownian_motion_generator.py is in the currnent path directory

from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend

In [2]:
#Fetch market prices for raw data
def coin_gecko_prices(coin, against='usd', days=1460):
    url = f'https://api.coingecko.com/api/v3/coins/{coin}/market_chart?vs_currency={against}&days={days}'.format(coin,against,days) 
    r = requests.get(url)
    df = pd.DataFrame(r.json()['prices'],columns=['unix',f'{coin}_{against}'])
    return df

df = coin_gecko_prices('bitcoin')
latest_market_price = df.tail(1)['bitcoin_usd'].values[0]
df


,unix,bitcoin_usd
0,1549929600000,3631.444540
1,1550016000000,3633.965047
2,1550102400000,3610.062273
3,1550188800000,3589.661830
4,1550275200000,3601.229191
...,...,...
1456,1675728000000,22786.483006
1457,1675814400000,23294.913648
1458,1675900800000,22947.507829
1459,1675987200000,21820.886508


In [3]:
#Generate random walks of prices

TIMESTEPS = 3
RUNS = 2

#Series should be mean stationary. So in this case of price series, convert them to log normal returns
df['btc_ln_return'] = np.log(df['bitcoin_usd'] / df['bitcoin_usd'].shift(1))
df = df.dropna(how='any')

#Approximate the distribution parameters of each series
#Here we will use a custom distribution for each: other options include 'normal' or 'laplace'
OU_params_btc = bmg.estimate_OU_params(df['btc_ln_return'].values, distribution_type='custom')

OU_params = (
    OU_params_btc,
)

correlations = df[
    ['btc_ln_return',
    ]
].corr().values[0]

OU_procs = bmg.simulate_corr_OU_procs(TIMESTEPS, OU_params, RUNS, rho=correlations)
OU_procs.shape


100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1972.40it/s]


(2, 3, 1)

In [4]:
#Turn the numpy array into a dataframe
runs,timesteps,procs = OU_procs.shape
OU_procs_arr = np.column_stack((np.repeat(np.arange(runs),timesteps),OU_procs.reshape(runs*timesteps,-1)))
walks = pd.DataFrame(OU_procs_arr,
                     columns=[
                         'run',
                         'btc_ln_return',
                     ])

walks['run'] = walks['run'].astype('int') + 1
walks['timestep'] = walks.groupby('run').cumcount() + 1

#Then turn this into a dictionary with keys of (run,timestep) to easily query the simulated return at a given run,timestep
walks = walks.set_index(['run','timestep']).to_dict('index')
walks


{(1, 1): {'btc_ln_return': 0.0012268365189861887},
 (1, 2): {'btc_ln_return': 0.007080742183728739},
 (1, 3): {'btc_ln_return': 0.006321716774943731},
 (2, 1): {'btc_ln_return': 0.0012268365189861887},
 (2, 2): {'btc_ln_return': 0.0068902324041814796},
 (2, 3): {'btc_ln_return': 0.0017360820613373704}}

Now it's time to setup our radCAD model, which will use the `walks` variable set above as the source of stochastic randomness for price movements.


In [122]:
system_params = {
    'trade_leverage_ratio': [2], #1 means no leverage. A floor is set in policy functions so 1 is smallest possible value
    'risk_level_liquidation': [1.1], #liquidated if Total Assets / (Borrowed Value + Accrued Interest) is < this
    'daily_borrowing_rate': [0.0002], #in APR. ie 0.0002 is equal to 0.02% per day, or 7.3% per year (0.02% * 365)
}

initial_state = {
    'asset_price': latest_market_price,
    'running_pnL': 0, #tracks cumulative profit and loss on the position
    'usd_borrowed': 0,
    'accrued_interest': 0,

}


In [123]:
def p_borrow_usd(params, substep, state_history, prev_state, **kwargs):
    borrow_usd = initial_state['asset_price'] * (max(1,params['trade_leverage_ratio']) - 1)
    return {'borrow_usd': borrow_usd}

def s_usd_borrowed(params, substep, state_history, prev_state, policy_input, **kwargs):
    return ('usd_borrowed', policy_input['borrow_usd'])


def p_accrue_interest(params, substep, state_history, prev_state, **kwargs):
    accrue_interest = prev_state['usd_borrowed'] * params['daily_borrowing_rate']
    return {'accrue_interest': accrue_interest}

def s_accrued_interest(params, substep, state_history, prev_state, policy_input, **kwargs):
    updated_accrued_interest = prev_state['accrued_interest'] + policy_input['accrue_interest']
    return ('accrued_interest', updated_accrued_interest)


def p_update_price(params, substep, state_history, prev_state, **kwargs):
    run = prev_state['run']
    timestep = prev_state['timestep']
    pct_price_change = np.exp(walks[(run, timestep)]['btc_ln_return'])
    return {'pct_price_change': pct_price_change}

def s_asset_price(params, substep, state_history, prev_state, policy_input, **kwargs):
    updated_asset_price = prev_state['asset_price'] * policy_input['pct_price_change']
    return ('asset_price', updated_asset_price)


def p_update_running_pnL(params, substep, state_history, prev_state, **kwargs):
    unleveraged_pnL = prev_state['asset_price'] - initial_state['asset_price']
    leveraged_pnL = unleveraged_pnL * max(1,params['trade_leverage_ratio'])
    print('LEVERAGED: ', leveraged_pnL, type(leveraged_pnL))
    return {'leveraged_pnL': leveraged_pnL}

def s_running_pnL(params, substep, state_history, prev_state, policy_input, **kwargs):
    running_pnL = policy_input['leveraged_pnL']
    print('RUNNING: ', running_pnL, type(running_pnL))
    return {'running_pnL', running_pnL}

# Next substep
#Update profit/loss (if prev state liquidated use prev state profit loss)
#Check if liquidated
    

In [124]:
state_update_blocks = [
    {
        'policies': {
            'borrow_funds': p_borrow_usd,
        },
        'variables': {
            'usd_borrowed': s_usd_borrowed,
        }
    },
    {
        'policies': {
            'accrue_interest': p_accrue_interest,
            'update_price': p_update_price,
        },
        'variables': {
            'accrued_interest': s_accrued_interest,
            'asset_price': s_asset_price,
        }
    },
    {
        'policies': {
            'update_pnL': p_update_running_pnL,
        },
        'variables': {
            'running_pnL': s_running_pnL,
        }
    },
]

In [125]:
model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)
experiment = Experiment(simulation)

# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine()#backend=Backend.PATHOS)

result = experiment.run()
df = pd.DataFrame(result)
# df = df[(df['substep'] == len(state_update_blocks)) | (df['timestep'] == 0)]
df

KeyError: 'Invalid state key running_pnL returned from state update function'